### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import utils
import plotly.graph_objs as go
import dash
from dash import dcc
from dash import html
import plots
from dash.dependencies import Input, Output
from dateutil import parser
%load_ext autoreload
%autoreload 2

### Parsing the Columns

In [2]:
name = 'shows'
bottom_channels = pd.read_csv(f"./{name}/bottom_{name}_vid.csv", index_col = 0)
bottom_channels = utils.parse_cols(bottom_channels)


top_channels = pd.read_csv(f"./{name}/top_{name}_vid.csv", index_col = 0)
top_channels = utils.parse_cols(top_channels)

Parser dropped 0 rows
Parser dropped 0 rows


### Describing the Data

In [ ]:
# print(top_channels.describe(percentiles = [.25, .5, .75, .9, .95, .99]))
# print(bottom_channels.describe(percentiles = [.25, .5, .75, .9, .95, .99]))

### Adding the popularity Column and combining the popular and unpopular dataframes

In [3]:
bottom_channels = utils.add_pop_unpop_col(bottom_channels, False)
top_channels = utils.add_pop_unpop_col(top_channels, True)

In [4]:
comb_vids = utils.combine_pop_unpop_df(top_channels, bottom_channels)

In [5]:
# comb_vids

### Removing the Outliers of the Data

In [6]:
# Remove outliers from the dataframe
cleaned_dataframe = utils.remove_outliers(comb_vids)
print(comb_vids.shape, cleaned_dataframe.shape)

(15154, 21) (13539, 21)


### Splitting the columns into 2 Chunks each having popular and unpopular channels on the base of mean of views

In [7]:
# chunklow_ch_names, chunkhigh_ch_names = utils.split_and_merge_by_views(comb_autos_info)
# chunklow_ch_names.channelName.tolist(), chunkhigh_ch_names.channelName.tolist()

In [8]:
summed_views_df, mean_views_popular, mean_views_unpopular, popular_below_mean, popular_above_mean, unpopular_below_mean, unpopular_above_mean = utils.pop_unpop_chunks(cleaned_dataframe)

above_mean_chunk = popular_above_mean + unpopular_above_mean
below_mean_chunk = popular_below_mean + unpopular_below_mean


print("Mean views of popular channels: ", mean_views_popular)
print("Mean views of unpopular channels: ", mean_views_unpopular)

print(f"Popular channel names with views above mean: {(popular_above_mean)}")
print(f"Popular channel names with views below mean: {(popular_below_mean)}")
print(f"UnPopular channel names with views above mean: {(unpopular_above_mean)}")
print(f"UnPopular channel names with views above mean: {(unpopular_below_mean)}")

Mean views of popular channels:  226271825.9
Mean views of unpopular channels:  45337442.0
Popular channel names with views above mean: ['ScarySnowmanTV', 'The Key of Awesome', 'kipkay']
Popular channel names with views below mean: ['A&E', 'DanceOn', 'Doctissimo', 'KBS Drama', 'MoosePlays', 'MrCreepyPasta', 'Reckless Comedy']
UnPopular channel names with views above mean: ['AnimationEpic', 'Zebra Nursery Rhymes - Kids Song and Cartoons']
UnPopular channel names with views above mean: ['Earth Touch', 'FUTURE INC.', 'My Damn Channel', 'Nine-Tales Recaps ', 'Peppers TV', 'RealityGamingTV', 'Sunrise', 'Televisa Puebla']


In [9]:
# # Run the first App to see the growth of views over the years for above mean chunk
# app = plots.dynamic_view_plots(cleaned_dataframe)
# app.run_server(debug=True, use_reloader=False, mode="inline", port=23000)

In [12]:
# percentiles = ['.25', '.5', '.75', '.9', '.95', '.99']
# app = plots.percentiles_plot(cleaned_dataframe, percentiles=percentiles)
# app.run_server(debug=True, use_reloader=False, mode="inline", port=23001)

In [10]:
means = utils.return_means_from_percentiles_for_given_years(cleaned_dataframe, percentiles=['0.95'], years = [2021, 2022, 2023], column = 'viewCount', year_column='publishingYear')

In [11]:
means.to_csv(f"{name}.csv")